In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('Quora_question_pairs.csv')

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
new_df = df.sample(30000)

In [5]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
new_df = new_df.dropna(subset='question2')

In [7]:
new_df.duplicated().sum()

np.int64(0)

In [8]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
326071,How do you quit smoking?,how to quit smoking
107191,What's is your wildest experience as a teenager?,How would the different Avatar in Avatar unive...
154335,Why are some Windows 10 updates not eligible f...,"My gf is 29, has $40k in debt, and makes $50k/..."
368893,Can a stolen phone be activated?,How do I find my stolen phone?
209166,Is it worth to pay a traffic fine if a driver ...,"If one moves between states, should one get a ..."


In [9]:
ques_df.isnull().sum()

question1    0
question2    0
dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
questions  = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features = 3000)
q1_arr , q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [11]:
temp_df1 = pd.DataFrame(q1_arr,index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr,index=ques_df.index)
temp_df = pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(30000, 6000)

In [12]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [13]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
326071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
107191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154335,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
368893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(temp_df.iloc[:,0:-1].values , temp_df.iloc[:,-1].values , test_size=0.2,random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.7523333333333333